# Math Module

The math submodule contians a couple of mathematical functions that are uesfull for computations.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tvsclib.identification as ident
import tvsclib.utils as utils
import tvsclib.math as math

import timeit
from tvsclib.approximation import Approximation


#create a rank-1 matrix
T = np.arange(0,25*5).reshape((-1,1))@np.arange(0,25*5).reshape((1,-1))
dims_in =  np.array([25]*5)
dims_out = np.array([25]*5)
system = ident.identify(T,dims_in,dims_out,epsilon=1e-14)
print(system)

# Frobenius Norm

If we want to compute the Frobenius norm
$$\|A\|_F = \sqrt{\sum_{i,j} A_{i,j}}$$
of the trasnfer matrix $T$ we can do this efficiently without explixitly computing the trasnfer matrix.
For this we use the formulation 
$$\|A\|_F = \sqrt{\mathrm{tr} ( T^\top T)}= \sqrt{\mathrm{tr} (T T^\top)}$$
where the the trace $\mathrm{tr}$ is the sum over the diagonal entries of the matrix.
This approach is described in [[1](#Rice_2010)].

Using this we can compute the product of system with its transposed using state space techniques.
As we only require the calcaulation of the diagonal elements we can use a recursive algorithm.

First we consider a causal system. This the product $T T^\top$ results in this combined structure:

<img src="Diagrams/Frobenius.svg" height="500">

Now we can denote the mapping form the state $x_k'$ to $x_k$ as $K_k$ with $x_k = K_k x_k'$. We can now express $K_k$ with the recursive relationship
$$k_k = A_nK_{k-1}A_n^\top + B_k B_k\top.$$
We start with $K_0$ is an empty matrix.
Therefore we can express the relation between $u_k$ and $y_k$ with $y_k = (C_k K_{n-1} C_k^\top + D_kD_k^\top)u_k$.
Using this we can compute the trace of $T T^\top$ by evaluationg the formula
$$\mathrm{tr}(T T^\top) = \sum_{k=0}^K \mathrm{tr}(C_k K_{n-1} C_k^\top)+\mathrm{tr}(D_kD_k^\top)$$

An analogous computation is possible for the anticausal system.

For a mixed system we can seperate the sum in the Frobenius norm into
$$\|T\|_F = \sqrt{\sum_{(i,j)}T_{i,j}^2}= \sqrt{\sum_{(i,j)\in \text{causal part}}T_{i,j}^2+\sum_{(i,j)\in \text{blockdiagonal part}}T_{i,j}^2+\sum_{(i,j)\in \text{anticausal part}}T_{i,j}^2}$$
This means that we can compute the squared norms seperateley and can later combine them into the combined norm.

In [ ]:
math.frobeniusnorm(system)

In [ ]:
np.linalg.norm(T,"fro")

In [ ]:
timeit.timeit(lambda:np.linalg.norm(system.to_matrix()), number=10)

In [ ]:
timeit.timeit(lambda:np.linalg.norm(T), number=10)

In [ ]:
timeit.timeit(lambda:math.frobeniusnorm(system), number=10)

## Singular values and Hankel norm

If we have a matrei $T$ and want to know the singular valus of the Hankel matricies we can use the function `extract_sigmas`.
As this depends on the the input and output dimensins we have to give them to the functions.

In [ ]:
math.extract_sigmas(T, dims_in,dims_out)[0][1]

We can also compute the Hankel norm

$$\|T\|_H = \sup_{i}\|T_i\|$$

This is the supremum over the spectral norm (the matrix 2-norm) of each individual Hankel matrix.


For details see: page 9f and 264ff in [[2](#Dewilde_1998)].

Now we compute the Hankelnorm of the matrix and compare it to the $\sigma$s obtained by the approxiamtion algorithm. We can see that biggest $\sigma$ is equal to the hankel norm

In [ ]:
math.hankelnorm(T,dims_in,dims_out)

If we have already created an system we can directly compute the Hankel norm using the approximation.

In [ ]:
approx =Approximation(system)
max([np.max(approx.sigmas_anticausal[i]) for i in range(len(approx.sigmas_anticausal))])
max([np.max(approx.sigmas_causal[i]) for i in range(len(approx.sigmas_anticausal))])

[1]:<a id='Rice 2010'></a> J. K. Rice. “Efficient Algorithms for Distributed Control: A Structured Matrix
Approach”. Ph.D. thesis. Technische Universiteit Delft, 2010. URL: https://repository.tudelft.nl/islandora/object/uuid%3A50460cc9-2aba-4db5-b602-5373bb0bc609

[2]:<a id='Dewilde_1998'></a> P. Dewilde and A.-J. van der Veen. Time-Varying Systems and Computations. Jan. 1, 1998. ISBN : 978-0-7923-8189-1. [DOI : 10.1007/978- 1- 4757-
2817-0](https://doi.org/10.1007/978-1-4757-2817-0)